# CS4TB3 Project Proposal
By group02 (Jessica Lei, Mingnan Su)

Overview
---
Our task for this project is to take a step further with Jupyter Notebook’s given functionalities by providing users with the ability to animate the execution of FSM. We will be implementing animations using the python library of turtle.

Firstly, let the user provide FSM as input, import the Turtle package and then draw the FSM diagram. Next, ask the user to provide input in the format of string, e.g."abc". We put the given input into FSM and decide whether it fits FSM. If it fits, we execute the input in FSM and animate the execution of FSM by coloring the different components of the diagram.

Furthermore, we could also provide the visualization of the algorithm for making FSM deterministic and equivalence.

Implementation
--

Firstly, user will provide the FSM in a format of string. 
```
first line: initial state
second line: {final states}
all subsequent lines: "source symbol -> target"
```

In [29]:
user_FSM = """
0
f
0 a → 1
1 b → 1
1 c → f
0 c → f
"""

Then the program will parse the string and store the value in class `FiniteStateMachine`.

In [34]:
class FiniteStateMachine:
    def __init__(self, T, Q, R, q0, F):
        self.T, self.Q, self.R, self.q0, self.F = T, Q, R, q0, F
    def __repr__(self):
        return str(self.q0) + '\n' + ' '.join(self.F) + '\n' + \
               '\n'.join(r[0] + ' ' + r[1] + ' → ' + r[2] for r in self.R)

def parseFSM(fsm: str) -> FiniteStateMachine:
    fsm = [line for line in fsm.split('\n') if line.strip() != '']
    q0 = fsm[0].split()[0] # first line: initialstate
    F = set(fsm[1].split()) # second line: finalstate, finalstate, ...
    R = set()
    for line in fsm[2:]: # all subsequent lines: "source symbol → target"
        l, r = line.split('→')
        R |= {(l.split()[0], l.split()[1], r.split()[0])}
    T = {r[1] for r in R}
    Q = {q0} | F | {r[0] for r in R} | {r[2] for r in R}
    return FiniteStateMachine(T, Q, R, q0, F)

def accepts(fsm: FiniteStateMachine, tau: str) -> bool:
    nxt = {(q, a): r for (q, a, r) in fsm.R}
    q = fsm.q0
    for t in tau:
        if (q, t) in nxt: 
            print(q,t)
            q = nxt[q, t]
        else: 
            return False
    return q in fsm.F

In [35]:
A0 = parseFSM(user_FSM)
print(A0.q0)
print(A0.R)
print(A0.F)
print(accepts(A0, 'abc'))

0
{('0', 'c', 'f'), ('0', 'a', '1'), ('1', 'b', '1'), ('1', 'c', 'f')}
{'f'}
0 a
1 b
1 c
True


In order to build from Jupyter notebook, we require the following configurations:
1. pip install ipyturtle
2. jupyter nbextension enable --py --sys-prefix ipyturtle
3. pip install numpy

Furthermore, user also required set the certain parameters in order to generate a good-looking diagram. For each transition, user need to define which way the arrow will point to, as well as the special arrow such as the loop transition.  

transition diagram syntax could be written as:
```
from [start] by [terminal] to [end] [straight|arc] distance
```

In the example above, user should define the transitions as follow:
```
from 0 by c to f arc 2
from 0 by a to 1 straight 1
from 1 by b to 1 arc 0
from 1 by c to f straight 1
```

Last but not least, the expected output should be the following:

In [108]:
import turtle as t
import numpy as np

#basic configurations
t.TurtleScreen._RUNNING = True
t.width(3)

#draw state
def drawState(name, final):
    t.penup()
    t.setx(pos[name][0])
    t.sety(pos[name][1])
    t.pendown()
    if not final:
        t.circle(30)
        t.write(name, True, align="center", font=("Arial",30,"normal"))
    else:
        t.circle(30)
        t.penup()
        t.sety(pos[name][1]+5)
        t.pendown()
        t.circle(25)
        t.write(name, True, align="center", font=("Arial",30,"normal"))

#draw transition
def drawArrow(start, end, terminal, arc, dist):
    if dist != 0:
        pos[end] = (pos[start][0]+160*dist, pos[start][1])
        if arc == 'straight':
            t.penup()
            t.setx(pos[start][0]+30)
            t.sety(pos[start][1]+30)
            t.pendown()
            t.forward(50*dist)
            t.write(terminal, True, align="center", font=("Arial",20,"normal"))
            t.forward(40*dist)
            #draw the arrow
            currPos = t.pos()
            t.setpos(currPos[0]-10,currPos[1]+5)
            t.setpos(currPos)
            t.setpos(currPos[0]-10,currPos[1]-5)
        if arc == 'arc':
            side = (np.abs(pos[start][0]) - np.abs(pos[end][0])) / 2
            radius = np.sqrt(3) * side * 0.75
            t.penup()
            t.setx(pos[start][0]+side)
            t.sety(pos[start][1]-side/2)
            t.pendown()
            t.circle(radius,50)
            t.penup()
            t.circle(radius,260)
            t.pendown()
            t.circle(radius,50)
            t.write(terminal, True, align="center", font=("Arial",20,"normal"))
            #draw the arrow
            currPos = t.pos()
            t.setpos(currPos[0]-10,currPos[1]+5)
            t.setpos(currPos)
            t.setpos(currPos[0]-10,currPos[1]-5)
    else:
        t.penup()
        t.setx(pos[start][0])
        t.sety(pos[start][1]+30)
        t.circle(40,35)
        t.pendown()
        t.circle(40,145)
        t.write(terminal, True, align="center", font=("Arial",20,"normal"))
        #draw the arrow
        currPos = t.pos()
        t.setpos(currPos[0]-10,currPos[1]+5)
        t.setpos(currPos)
        t.setpos(currPos[0]-10,currPos[1]-5)
        t.setpos(currPos)
        t.circle(40,130)
        t.penup()
        t.circle(40,50)

#draw FSM
#state position map
pos = {}
t.speed(0)

#initial state 0
pos['0'] = (-400,0)
drawState('0', False)

#from 0 by c to f arc 2
#final state f
drawArrow('0', 'f', 'c', 'arc', 2)
drawState('f', True)

#from 0 by a to 1 straight 1
drawArrow('0', '1', 'a', 'straight', 1)
drawState('1', False)

#from 1 by b to 1 arc 0
drawArrow('1', '1', 'b', 'arc', 0)

#from 1 by c to f straight 1
#final state f
drawArrow('1', 'f', 'c', 'straight', 1)

#animate execution of abc
t.speed('normal')
t.pencolor('red')
#0 a
drawState('0', False)
drawArrow('0', '1', 'a', 'straight', 1)

#1 b
drawState('1', False)
drawArrow('1', '1', 'b', 'arc', 0)

#1 c
drawArrow('1', 'f', 'c', 'straight', 1)
drawState('f', True)

#end turtle
t.done()

Documentation
---
This project will be documented and demostrated by Jupyter notebooks throughout the development period. We hope to learn more about the execution and algorithms of FSM by completing this task. Furthermore, it also helps us to consolidate our knowledge on the topics and materials learned throughout the course.

Resources
---
`2. Regular Languages` Lecture Notes By Emil Sekerinski, McMaster University, January 2020   
`Python Turtle` Library   
`Python Numpy` Library

Division of Work
---
|`Team Member`    |`Task`  |
|:---------|--------|
|`Jessia`  | `poster design, generalized turtle code` |
|`Mingnan` | `poster content, turtle animation code, FSM parsing algorithms` |

Weekly Schedule
---
|`Deadline`    |`Task`  |
|:---------|--------|
|`Mar. 9`  | `turtle animation code` |
|`Mar. 16` | `generalized turtle code` |
|`Mar. 23` | `utilize turtle code into FSM parsing` |
|`Mar. 31` | `finalize project and bug fixes` |
|`Apr. 7`  | `poster design and content` |